In [1]:
# Standard imports
import numpy as np
import pandas as pd
import csv
import math
import matplotlib.pyplot as plt

In [2]:
def csv2ll(file):
    with open(file) as f:
        reader = csv.reader(f)
        return list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

In [3]:
data = csv2ll(r"fakedata.csv")

In [4]:
print(data)

[['1', '2', '3', '4', '5', '6', '6', '7', '7', '8', '8', '9', '9', '9', '00', '', '-', '', '', '1', '2', '1', '2', '22', '3', '3', '']]
